In [1]:
import cv2
import numpy as np

# Load YOLO Model
config_path = "C:/Users/pooja/Downloads/yolov3.cfg"  # Path to YOLO config file
weights_path = "C:/Users/pooja/Downloads/yolov3.weights"  # Path to YOLO weights file
net = cv2.dnn.readNet(weights_path, config_path)

In [2]:
# Load and optionally resize the image
image_path = "C:\\Users\\pooja\\Downloads\\key.jpg"  # Path to the image
img = cv2.imread(image_path)

# Optionally resize the image to enhance detection
scale_factor = 1.5  # Adjust this value if necessary
img = cv2.resize(img, (int(img.shape[1] * scale_factor), int(img.shape[0] * scale_factor)))

height, width, _ = img.shape

In [3]:
# Prepare the image for YOLO by converting it into a blob
blob = cv2.dnn.blobFromImage(img, 0.00392, (416, 416), (0, 0, 0), True, crop=False)
net.setInput(blob)

In [4]:
# Get output layer names
layer_names = net.getLayerNames()
# Adjust to handle the unconnected layers properly
output_layers = [layer_names[i - 1] for i in net.getUnconnectedOutLayers().flatten()]

In [5]:
# Perform a forward pass to get predictions
outputs = net.forward(output_layers)


In [6]:
# Initialize lists for bounding boxes, confidences, and class IDs
boxes = []
confidences = []
class_ids = []

In [7]:
# Process the outputs and apply Non-Max Suppression (NMS)
for output in outputs:
    for detection in output:
        scores = detection[5:]  # Skip the first 5 values (bounding box coordinates)
        class_id = np.argmax(scores)  # Get the index of the highest score
        confidence = scores[class_id]
        
        if confidence > 0.1:  # Lowered confidence threshold to detect smaller objects
            center_x = int(detection[0] * width)
            center_y = int(detection[1] * height)
            w = int(detection[2] * width)
            h = int(detection[3] * height)
            x = int(center_x - w / 2)
            y = int(center_y - h / 2)
            boxes.append([x, y, w, h])
            confidences.append(float(confidence))
            class_ids.append(class_id)

# Apply Non-Max Suppression (NMS) to remove overlapping boxes
indices = cv2.dnn.NMSBoxes(boxes, confidences, score_threshold=0.1, nms_threshold=0.4)

In [8]:
# Print detected objects and their confidence levels
if len(indices) > 0:
    for i in indices.flatten():
        print(f"Detected Object {class_ids[i]} with Confidence {confidences[i]:.2f}")
        x, y, w, h = boxes[i]
        cv2.rectangle(img, (x, y), (x + w, y + h), (0, 255, 0), 2)
        cv2.putText(img, "Object detected", (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
else:
    print("No objects detected.")

Detected Object 76 with Confidence 0.58


In [ ]:
# Display the resulting image with bounding boxes
cv2.imshow("Detected Objects", img)
cv2.waitKey(0)  # Wait until any key is pressed
cv2.destroyAllWindows()  # Close the image window

In [17]:
# Estimate the dimensions of the detected object (using a reference object)
if len(indices) > 0:  # Ensure that NMS found at least one object
    for i in indices.flatten():  # Flatten the indices to avoid scalar index issues
        # Access the first detected object's bounding box
        x, y, w, h = boxes[i]  # w = width in pixels, h = height in pixels

        ref_width = 10  # cm (the known width of a reference object in your image)
        pixel_per_metric = w / ref_width  # Calculate pixels per centimeter
        real_width = w / pixel_per_metric
        real_height = h / pixel_per_metric

        # Assuming a length-to-width ratio or using another reference
        length_to_width_ratio = 1.5  # Example: If length is 1.5 times the width
        real_length = real_width * length_to_width_ratio

        print(f"Estimated Object Dimensions: Width = {real_width:.2f} cm, Height = {real_height:.2f} cm, Length = {real_length:.2f} cm")
else:
    print("No objects detected to estimate dimensions.")

Estimated Object Dimensions: Width = 10.00 cm, Height = 12.94 cm, Length = 15.00 cm
